In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem
import nest_asyncio
nest_asyncio.apply()
# setting up api key
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
import wikipedia
import openmeteo_requests
import requests_cache
# from retry_requests import retry
import requests
from agents.extensions.visualization import draw_graph

import json 
import requests
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")


from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

from tools import flight_search, yelp_search, hotels_search, wikipedia_search, get_weather
from pydantic import BaseModel
from models import first_agent_output
from agents.model_settings import ModelSettings
from data_models.flight_data_models import FlightSearchResults

from travel_agents import question_rewrite_agent, flight_agent
from functions import flight_search

END OF THE SCRIPT


In [30]:
main_question = """I want to travel from Ankara to Istanbul. The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
I also need a hotel to stay. I do not like smell of cigarette. If possible, I want to see the rooms where smoking is not allowed. 
In addition to that, I like eating sea food. I guess istanbul is very rich city interms of sea restaurants""".strip()

question_rewrite_agent_output = Runner.run_sync(question_rewrite_agent,main_question)




In [3]:
question_rewrite_agent_output.final_output

FinalModel(flight_reqs=FlightOutput(departure_id='Ankara', arrival_id='Amsterdam', outbound_data='2025-05-05', return_data='2025-05-11'), yelp_reqs=YelpOutput(search_term='sea food', location='Istanbul'), hotel_reqs=HotelOutput(query='Amsterdam', check_in_date='2025-05-05', check_out_date='2025-05-11', gl='non-smoking'))

In [4]:
question_rewrite_agent_output.final_output

FinalModel(flight_reqs=FlightOutput(departure_id='Ankara', arrival_id='Amsterdam', outbound_data='2025-05-05', return_data='2025-05-11'), yelp_reqs=YelpOutput(search_term='sea food', location='Istanbul'), hotel_reqs=HotelOutput(query='Amsterdam', check_in_date='2025-05-05', check_out_date='2025-05-11', gl='non-smoking'))

In [5]:
question_rewrite_agent_output.final_output.flight_reqs.return_data

'2025-05-11'

In [6]:
# flight_results = Runner.run_sync(flight_agent,main_question)

In [7]:
x = flight_search(
    departure_id= "ESB",
    arrival_id='AMS',
    # departure_id=question_rewrite_agent_output.final_output.flight_reqs.departure_id, 
    # arrival_id= question_rewrite_agent_output.final_output.flight_reqs.arrival_id ,
    outbound_data= question_rewrite_agent_output.final_output.flight_reqs.outbound_data, 
    return_date = question_rewrite_agent_output.final_output.flight_reqs.return_data
    )

# flight_results = FlightSearchResults.parse_obj(x)  # Use parse_obj with dict


In [8]:
flight_results = FlightSearchResults.parse_obj(x)  # Use parse_obj with dict

C:\Users\user\AppData\Local\Temp\ipykernel_32348\897696582.py:1: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  flight_results = FlightSearchResults.parse_obj(x)  # Use parse_obj with dict


In [9]:
flight_results

FlightSearchResults(search_metadata=SearchMetadata(id='67fbfc57cbc42b6586a0054f', status='Success', json_endpoint='https://serpapi.com/searches/12487f80c39b39af/67fbfc57cbc42b6586a0054f.json', created_at='2025-04-13 18:03:03 UTC', processed_at='2025-04-13 18:03:03 UTC', google_flights_url='https://www.google.com/travel/flights?hl=en&gl=us&curr=USD&tfs=CBwQAhoeEgoyMDI1LTA1LTA1agcIARIDRVNCcgcIARIDQU1TGh4SCjIwMjUtMDUtMTFqBwgBEgNBTVNyBwgBEgNFU0JCAQFIAXABmAEB&tfu=EgIIAQ', raw_html_file='https://serpapi.com/searches/12487f80c39b39af/67fbfc57cbc42b6586a0054f.html', prettify_html_file='https://serpapi.com/searches/12487f80c39b39af/67fbfc57cbc42b6586a0054f.prettify', total_time_taken=0.57), search_parameters=SearchParameters(engine='google_flights', hl='en', gl='us', departure_id='ESB', arrival_id='AMS', outbound_date='2025-05-05', return_date='2025-05-11', currency='USD', deep_search=True), best_flights=[BestFlight(flights=[Flight(departure_airport=AirportInfo(name='Ankara Esenboga Airport', i

In [10]:

flight_info = ""
c = 0
for j in flight_results.best_flights:
  for i, flight in enumerate(j.flights, start=1):
      c += 1
      x= f"""
  Flight {c}:
    Departure: {flight.departure_airport.name} ({flight.departure_airport.id}) at {flight.departure_airport.time}
    Arrival: {flight.arrival_airport.name} ({flight.arrival_airport.id}) at {flight.arrival_airport.time}
    Duration: {flight.duration} minutes
    Airline: {flight.airline}
    Airplane: {flight.airplane}
    Class: {flight.travel_class}
    Flight Number: {flight.flight_number}
    Legroom: {flight.legroom}
    Extensions: {', '.join(flight.extensions)}
  """
      flight_info += x


for j in flight_results.other_flights:
  for i, flight in enumerate(j.flights, start=1):
      c += 1
      x= f"""
  Flight {c}:
    Departure: {flight.departure_airport.name} ({flight.departure_airport.id}) at {flight.departure_airport.time}
    Arrival: {flight.arrival_airport.name} ({flight.arrival_airport.id}) at {flight.arrival_airport.time}
    Duration: {flight.duration} minutes
    Airline: {flight.airline}
    Airplane: {flight.airplane}
    Class: {flight.travel_class}
    Flight Number: {flight.flight_number}
    Legroom: {flight.legroom}
    Extensions: {', '.join(flight.extensions)}
  """
      flight_info += x


In [17]:
print(flight_info)


  Flight 1:
    Departure: Ankara Esenboga Airport (ESB) at 2025-05-05 19:20
    Arrival: Antalya Airport (AYT) at 2025-05-05 20:35
    Duration: 75 minutes
    Airline: AJet
    Airplane: None
    Class: Economy
    Flight Number: VF 4008
    Legroom: 29 in
    Extensions: Below average legroom (29 in), Carbon emissions estimate: 55 kg
  
  Flight 2:
    Departure: Antalya Airport (AYT) at 2025-05-05 22:45
    Arrival: Amsterdam Airport Schiphol (AMS) at 2025-05-06 01:55
    Duration: 250 minutes
    Airline: TUI fly Netherlands
    Airplane: None
    Class: Economy
    Flight Number: OR 3016
    Legroom: 30 in
    Extensions: Average legroom (30 in), In-seat USB outlet, On-demand video, Carbon emissions estimate: 170 kg
  
  Flight 3:
    Departure: Ankara Esenboga Airport (ESB) at 2025-05-05 04:25
    Arrival: Sabiha Gökçen International Airport (SAW) at 2025-05-05 05:35
    Duration: 70 minutes
    Airline: AJet
    Airplane: Airbus A321
    Class: Economy
    Flight Number: VF 30

In [23]:
f_agent = Agent(name="Flight Assistant agent",
              instructions=f"You are helpful flight asisstants try to answer the questions according to: \n\n{flight_info}. You are not allowed to recommend or answer any other thing than flights",
            #   model="gpt-4o-mini",
              model = 'gpt-4o',
              model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*8),
              )

In [28]:
result = Runner.run_sync(f_agent, main_question+" bring me cheapest flight, also tell me how much does that cost?")
print(result.final_output)

I can help you with the flights from Ankara to Amsterdam, but I can't assist with hotel bookings or restaurant recommendations. Here's the cheapest flight option for your outbound journey:

### Outbound Flight (Ankara to Amsterdam on 2025-05-05):
1. **Flight 25**: 
   - Departure: Ankara Esenboga Airport (ESB) at 08:00
   - Arrival: Istanbul Airport (IST) at 09:25
   - Duration: 85 minutes
   - Airline: Turkish Airlines
   - Flight Number: TK 2117

2. **Flight 26**:
   - Departure: Istanbul Airport (IST) at 10:40
   - Arrival: Amsterdam Airport Schiphol (AMS) at 13:15
   - Duration: 215 minutes
   - Airline: Turkish Airlines
   - Flight Number: TK 1957

### Return Flight (Amsterdam to Ankara on 2025-05-11):
Unfortunately, I don't have information on return flights for that date. You may need to check with airlines directly for the return journey.

### Cost:
I don't have access to pricing information. Please check with the airline or a travel booking site for the cost of these flights.


In [29]:
result = Runner.run_sync(f_agent, "list back all the flights")
print(result.final_output)

Here is a list of all the flights:

1. **Flight 1:**
   - **Departure:** Ankara Esenboga Airport (ESB) at 2025-05-05 19:20
   - **Arrival:** Antalya Airport (AYT) at 2025-05-05 20:35
   - **Duration:** 75 minutes
   - **Airline:** AJet
   - **Flight Number:** VF 4008

2. **Flight 2:**
   - **Departure:** Antalya Airport (AYT) at 2025-05-05 22:45
   - **Arrival:** Amsterdam Airport Schiphol (AMS) at 2025-05-06 01:55
   - **Duration:** 250 minutes
   - **Airline:** TUI fly Netherlands
   - **Flight Number:** OR 3016

3. **Flight 3:**
   - **Departure:** Ankara Esenboga Airport (ESB) at 2025-05-05 04:25
   - **Arrival:** Sabiha Gökçen International Airport (SAW) at 2025-05-05 05:35
   - **Duration:** 70 minutes
   - **Airline:** AJet
   - **Flight Number:** VF 3011

4. **Flight 4:**
   - **Departure:** Sabiha Gökçen International Airport (SAW) at 2025-05-05 08:45
   - **Arrival:** Amsterdam Airport Schiphol (AMS) at 2025-05-05 11:30
   - **Duration:** 225 minutes
   - **Airline:** AJet
  